In [1]:
# Base imports to avoid headaches
from pprint import pprint

import numpy as np
import plotly.express as px

from main.abae.config import ABAEManagerConfig
import json
import pandas as pd
from main.abae.model_manager import ABAEManager

In [2]:
# Base corpus references for file
corpus = "../dataset/output/default/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default/pre_processed.310k.test.csv"

In [3]:
# Now that I did some hyperparameters tuning we try a model and see the results

In [4]:
# Results of hyperparameter tuning are stored here
file_path = "./output/config/abae_configurations_results.json"

configs = [
    pd.DataFrame(json.load(open(file_path))).at[11, 'config'],
    pd.DataFrame(json.load(open(file_path))).at[20, 'config'],

    # These two perform worse but have a lower variance which might result in a more robust solution
    pd.DataFrame(json.load(open(file_path))).at[14, 'config'],
    pd.DataFrame(json.load(open(file_path))).at[15, 'config'],
]

In [8]:
runs = []
# Create runs configurations
for i in range(len(configs)):
    config = ABAEManagerConfig.from_configuration(f"final_full{i}", configs[i])
    runs.append(dict(config=config, results=[], model=None))

In [ ]:
# Train the runs
tops = list(range(1, 30))  # Measure more infos to see the trend
tops.reverse()

for run in runs:
    manager = ABAEManager.from_config(run["config"], corpus, override=True)
    manager.train(df=corpus, verbose=2)  # 2 Is epoch only

    run['results'] = manager.evaluate(tops=tops, test_corpus=test_corpus_path)
    run['model'] = manager.get_compiled_model(load_existing=True)  # Load the compiled model

    pprint(run['results'])  # Show run results

Pandas Apply:   0%|          | 0/233384 [00:00<?, ?it/s]

D:\PycharmProjects\nlp-course-project\.venv\Lib\site-packages\gensim\models\keyedvectors.py:551: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(
D:\PycharmProjects\nlp-course-project\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Das System kann die angegebene Datei nicht finden
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "D:\PycharmProjects\nlp-course-project\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\jacop\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in 

Creating new Aspect embedding model

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/233384 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 5614(2.4054776677064407% of ds).
Generating a new compiled model from scratch
Training is starting:
Epoch 1/9
912/912 - 259s - 284ms/step - loss: 5.5125 - max_margin_loss: 5.5109
Epoch 2/9
912/912 - 259s - 284ms/step - loss: 4.1676 - max_margin_loss: 4.1667
Epoch 3/9


In [ ]:
[pprint(run['results']) for run in runs]  # Show run results

In [ ]:
# Compare results
structured_data = {'config': [], 'coherence': [], 'top': []}
config_num = 1
for run in runs:
    print(f"Results for run {run['config']}:\n {run['results']}")
    print(f"Overall AVG: {np.mean(run['results'])} and VAR: {np.var(run['results'])}")

    for coherence, top in zip(run['results']['coherence'], run['results']['top']):
        structured_data['config'].append(str(config_num))  # Id for color

        structured_data['coherence'].append(coherence)
        structured_data['top'].append(top)

    config_num += 1

In [ ]:
data = pd.DataFrame(structured_data)

# Plot the moving coherence
px.line(data, x="top", y="coherence", color='config', title='Hyperparameter Tuning Results').show()

Now see if we can map aspects to best ones:

In [ ]:
best = 2

In [ ]:
manager = ABAEManager.from_config(runs[best], corpus)
manager.get_compiled_model(load_existing=True)
ev_processor = manager.make_ev_processor(test_corpus_path)

# Get the aspect words to map
aspects = ev_processor.get_aspects(20)

In [ ]:
aspects

It somehow is weird that the 80k model performs much better, let's make a comparison:

In [ ]:
# Fai 80k e fai test su ds di test da 300k dopo aver tolto elementi del 80k train

In [ ]:
# Ora hai fatto il confronto ma modello è migliore? Guardiamo coherence per topic
# Si scopre data is unbalanced e ci sono schemi difficili da riconoscere
# Se non li usiamo per il task e consideriamo coerente tutto cio in miscellanea il nostro modello
# potremmo volerlo pesare solo su labeling pertinente